<a href="https://colab.research.google.com/github/Guylord/APHRC_DSE_Insprire_Hackathon_2024/blob/main/APHRC_DSE_Insprire_Hackathon_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [13]:
hdss_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_hdss_v3.csv')
facility_df = pd.read_csv('/content/drive/MyDrive/APHRC-DSE Inspire Hackathon 2024/synthetic_facility_v3.csv')

print('HDSS INFO')
hdss_df.info()
print('\n\nFACILITY INFO')
facility_df.info()

HDSS INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4115 entries, 0 to 4114
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   recnr       4115 non-null   int64  
 1   firstname   4115 non-null   object 
 2   lastname    4031 non-null   object 
 3   petname     2894 non-null   object 
 4   dob         4115 non-null   object 
 5   sex         4115 non-null   int64  
 6   nationalid  0 non-null      float64
 7   hdssid      4115 non-null   object 
 8   hdsshhid    4115 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 289.5+ KB


FACILITY INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recnr       2902 non-null   int64 
 1   firstname   2902 non-null   object
 2   lastname    2835 non-null   object
 3   petname     2511 non-null   object
 4   do

# *FACILITY dataset*

In [4]:
facility_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020


In [17]:
facility_df['dob'] = pd.to_datetime(facility_df['dob'])
facility_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   recnr       2902 non-null   int64         
 1   firstname   2902 non-null   object        
 2   lastname    2835 non-null   object        
 3   petname     2511 non-null   object        
 4   dob         2902 non-null   datetime64[ns]
 5   sex         2902 non-null   int64         
 6   nationalid  2902 non-null   object        
 7   patientid   2902 non-null   int64         
 8   visitdate   2902 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 204.2+ KB


In [5]:
for column in facility_df.select_dtypes('object').columns:
  facility_df[column] = facility_df[column].str.strip()

In [ ]:
facility_df.duplicated().sum()

In [35]:
facility_df.duplicated(subset=['firstname', 'lastname', 'dob'], keep=False).sum()

570

In [33]:
facility_df[facility_df.duplicated(subset=['firstname', 'lastname', 'dob'], keep=False)].sort_values('dob').head(10)

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
1517,1519,Rayan,Sowobi,Nalubega,1930-09-23,1,N_ID_9515,3588,25-05-2019
1360,1362,Rayan,Sowobi,Nalubega,1930-09-23,1,N_ID_9044,3431,04-08-2018
1432,1434,Hassan,Isa,Deo,1931-05-24,1,N_ID_9260,3503,15-09-2022
1745,1747,Hassan,Isa,Deo,1931-05-24,1,N_ID_10202,3816,22-02-2021
733,735,Derick,Falidi,Akili,1931-07-06,1,N_ID_7175,2805,04-02-2021
545,547,Derick,Falidi,Akili,1931-07-06,1,N_ID_6605,2617,12-11-2022
1595,1597,Nakidodo,Nakagolo,Nambi,1932-06-12,2,N_ID_9749,3666,06-01-2021
644,646,Nakidodo,Nakagolo,Nambi,1932-06-12,2,N_ID_6902,2716,26-02-2021
677,679,Sawuya,Namukose,Amutu,1932-11-30,2,N_ID_7001,2749,09-02-2020
155,157,Sawuya,Namukose,Amutu,1932-11-30,2,N_ID_5426,2227,25-10-2019


In [41]:
facility_df.duplicated(subset='nationalid', keep=False).sum()

0



It's evident that some records have the same entries for 'firstname,' 'lastname,' 'petname,' and 'dob,' implying that they refer to the same entity but with different 'nationalid' entries during various visits to the facility. Furthermore, there are no duplicate entries for 'nationalid' even on different visit dates. This suggests that individuals visiting the facility did not retain a single 'nationalid,' rendering 'nationalid' unusable as a unique identifier


In [8]:
facility_df.isna().sum()

recnr           0
firstname       0
lastname       67
petname       391
dob             0
sex             0
nationalid      0
patientid       0
visitdate       0
dtype: int64

In [ ]:
facility_df['sex'].nunique()

2

# *HDSS dataset*

In [ ]:
hdss_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid
0,1,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001
1,2,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002
2,3,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003
3,4,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004
4,5,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005


In [ ]:
for column in hdss_df.select_dtypes('object').columns:
  hdss_df[column] = hdss_df[column].str.strip()

In [ ]:
hdss_df.duplicated().sum()

0

In [ ]:
hdss_df['sex'].nunique()

2

In [ ]:
import recordlinkage

In [ ]:
indexer = recordlinkage.Index()
indexer.block('dob')
pairs = indexer.index(facility_df, hdss_df)

In [ ]:
compare = recordlinkage.Compare()

compare.exact('dob', 'dob', label = 'dob')
compare.exact('sex', 'sex', label = 'sex')

compare.string('fullname', 'fullname', threshold = 0.85, label='fullname')
compare.string('petname', 'petname', threshold=0.85, label='petname')

potential_matches=compare.compute(pairs, facility_df, hdss_df)
potential_matches

,,dob,sex,fullname,petname
0,2740,1,1,0.0,0.0
1,2612,1,1,1.0,0.0
3,1178,1,1,1.0,0.0
6,2901,1,1,1.0,1.0
7,2950,1,1,1.0,1.0
...,...,...,...,...,...
2884,2126,1,1,1.0,0.0
2892,3320,1,1,1.0,1.0
2893,691,1,1,1.0,1.0
2898,3446,1,1,1.0,0.0


In [ ]:
matches=potential_matches[potential_matches.sum(axis=1)>=2]
matches

,,dob,sex,fullname,petname
1,2612,1,1,1.0,0.0
3,1178,1,1,1.0,0.0
6,2901,1,1,1.0,1.0
7,2950,1,1,1.0,1.0
2409,433,1,1,1.0,1.0
...,...,...,...,...,...
2884,2126,1,1,1.0,0.0
2892,3320,1,1,1.0,1.0
2893,691,1,1,1.0,1.0
2898,3446,1,1,1.0,0.0


In [ ]:
duplicate_rows=matches.index.get_level_values(1)
facility_duplicates=facility_df[facility_df.index.isin(duplicate_rows)]
facility_new = facility_df[~facility_df.index.isin(duplicate_rows)]
facility_hdss =hdss_df.append(facility_new)
facility_hdss

<ipython-input-110-3dd5d0163176>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  facility_hdss =hdss_df.append(facility_new)


,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
0,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001,Zaina Hanifa,NaN,NaN
1,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002,Godfrey Maganda,NaN,NaN
2,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003,Kasim Ngobi,NaN,NaN
3,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004,Esther,NaN,NaN
4,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005,Sumaya Swabula,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2895,Swabiru,Aluvin,Najjuma,19-05-2014 00:00,1,N_ID_13670,NaN,NaN,Swabiru Aluvin,4959.0,16-10-2022
2897,Madina,Nakagolo,Nkima,28-07-1955 00:00,2,N_ID_13676,NaN,NaN,Madina Nakagolo,4961.0,17-08-2021
2898,Namulondo,Namugabwe,NaN,14-09-1933 00:00,2,N_ID_13679,NaN,NaN,Namulondo Namugabwe,4962.0,21-06-2019
2899,Ziriya,Kauma,Kato,11-11-1963 00:00,2,N_ID_13682,NaN,NaN,Ziriya Kauma,4963.0,27-12-2018


In [ ]:
facility_hdss.shape

(6395, 11)

In [ ]:
facility_hdss.duplicated().sum()

0

In [ ]:
facility_hdss.duplicated(subset=['fullname', 'petname', 'dob', 'sex']).sum()

879

In [ ]:
facility_hdss.drop_duplicates(subset=['fullname', 'petname', 'dob', 'sex'], keep='first')
facility_hdss

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
0,Zaina,Hanifa,Ula,22-09-1930 00:00,2,NaN,I20001,HH100001,Zaina Hanifa,NaN,NaN
1,Godfrey,Maganda,Mukama,15-07-1934 00:00,1,NaN,I20002,HH100002,Godfrey Maganda,NaN,NaN
2,Kasim,Ngobi,Galabuzi,03-03-1983 00:00,1,NaN,I20003,HH100003,Kasim Ngobi,NaN,NaN
3,Esther,NaN,Inara,30-07-1968 00:00,2,NaN,I20004,HH100004,Esther,NaN,NaN
4,Sumaya,Swabula,NaN,13-12-1930 00:00,2,NaN,I20005,HH100005,Sumaya Swabula,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2895,Swabiru,Aluvin,Najjuma,19-05-2014 00:00,1,N_ID_13670,NaN,NaN,Swabiru Aluvin,4959.0,16-10-2022
2897,Madina,Nakagolo,Nkima,28-07-1955 00:00,2,N_ID_13676,NaN,NaN,Madina Nakagolo,4961.0,17-08-2021
2898,Namulondo,Namugabwe,NaN,14-09-1933 00:00,2,N_ID_13679,NaN,NaN,Namulondo Namugabwe,4962.0,21-06-2019
2899,Ziriya,Kauma,Kato,11-11-1963 00:00,2,N_ID_13682,NaN,NaN,Ziriya Kauma,4963.0,27-12-2018


In [ ]:
facility_hdss.shape

(6395, 11)

In [ ]:
duplicates_mask = facility_hdss.duplicated(subset=['fullname', 'petname', 'dob', 'sex'])

# Filter the DataFrame to keep only the duplicate rows
duplicate_rows = facility_hdss[duplicates_mask]

# Display the result
print("Duplicate Rows:")
print(duplicate_rows)


Duplicate Rows:
      firstname   lastname    petname               dob  sex  nationalid  \
1        Gloria    Rashida        NaN  11-07-1993 00:00    2  N_ID_11861   
6     Namutamba     Sumaya       Bria  18-02-1997 00:00    2  N_ID_11876   
10       Nasiri    Kirunda      Ebitu  02-06-1959 00:00    1   N_ID_5015   
16       Hajara   Namusubo        NaN  02-12-1955 00:00    2   N_ID_5033   
29       Aminsi       Ibra       Jose  22-12-2000 00:00    1   N_ID_5072   
...         ...        ...        ...               ...  ...         ...   
2891     Safalu        NaN       Atek  07-04-2025 00:00    1  N_ID_13658   
2893     Ukasha  Gubayanga    Kulubya  24-12-1987 00:00    1  N_ID_13664   
2898  Namulondo  Namugabwe        NaN  14-09-1933 00:00    2  N_ID_13679   
2899     Ziriya      Kauma       Kato  11-11-1963 00:00    2  N_ID_13682   
2900      Faizo    Buyinza  Ssenyonjo  09-06-1956 00:00    1  N_ID_13685   

     hdssid hdsshhid             fullname  patientid   visitdate  
1   

In [ ]:
facility_hdss[facility_hdss['fullname'] == 'Gloria Rashida']

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
2612,Gloria,Rashida,NaN,11-07-1993 00:00,2,NaN,I22613,HH102613,Gloria Rashida,NaN,NaN
1,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,NaN,NaN,Gloria Rashida,2079.0,14-12-2022
2651,Gloria,Rashida,Bugingo,21-06-1993 00:00,2,N_ID_12956,NaN,NaN,Gloria Rashida,4721.0,12-07-2022


In [1]:
pip install sqlalchemy